# Usage Examples

Here basic usage of the `mt_metadata` module are demonstrated.

## Base Class

`mt_metadata.base.Base` is the base for which all metadata objects are built upon.  `Base` provides convenience filters to input and output metadata in different formats XML, JSON, Python dictionary, Pandas Series.  It also provides functions to help the user understand what's inside.

In [1]:
from mt_metadata.base import Base

2022-01-10T12:42:49 [line 157] numexpr.utils._init_num_threads - INFO: NumExpr defaulting to 8 threads.


In [2]:
b = Base()

### Methods of Base

Methods of Base include `to/from_[json, dict, series, xml]` which allows the user to input and output data in various standard formats including

- JSON
- XML
- standard Python dictonary
- Pandas.Series

Also included are methods to get information about the standards and attributes included in the metadata object, and the ability to add new attributes.  Finally, methods to `get` and `set` an attribute by a compound name like `location.declination.value`.  

In [3]:
print("\n\t".join(["Methods:"] + [func for func in dir(b) if callable(getattr(b, func)) and not func.startswith("_")]))

Methods:
	add_base_attribute
	attribute_information
	from_dict
	from_json
	from_series
	from_xml
	get_attr_from_name
	get_attribute_list
	set_attr_from_name
	to_dict
	to_json
	to_series
	to_xml
	update


#### Add attributes

You can add attibutes to an existing metadata object.  All you need is to add a standards dictionary that describes the new attribute.

Here we will add an extra attribute for temperature.  We will allow it to only have two options 'ambient' or 'air'.  It will be a `string` but is not required.  

In [4]:
extra = {
    'type': str,
    'style': 'controlled vocabulary',
    'required': False,
    'units': 'celsius',
    'description': 'local temperature',
    'alias': ['temp'],
    'options': [ 'ambient', 'air'],
    'example': 'ambient',
    'default': None
}

In [5]:
b.add_base_attribute("temperature", "ambient", extra)

#### The `__repr__`

The base class `__repr__` is represented by the JSON representation of the object. 

In [6]:
b

{
    "base": {
        "temperature": "ambient"
    }
}

#### The `__str__`

The `__str__` of the class is a printed list

In [7]:
print(b)

base:
	temperature = ambient


#### Attribute Information and List

There is also a convenience method to get attribute information.

In [8]:
b.get_attribute_list()

['temperature']

In [9]:
b.attribute_information()

temperature:
	alias: ['temp']
	default: None
	description: local temperature
	example: ambient
	options: ['ambient', 'air']
	required: False
	style: controlled vocabulary
	type: <class 'str'>
	units: celsius


In [10]:
b.attribute_information("temperature")

temperature:
	alias: ['temp']
	default: None
	description: local temperature
	example: ambient
	options: ['ambient', 'air']
	required: False
	style: controlled vocabulary
	type: <class 'str'>
	units: celsius


## Validation

Validation of the attribute is the most important part of having a separate module for the metadata.  The validation processes

1. First assures the `type` is the correct type prescribed by the metadata.  For example in the above example the prescribed data type for `temperature` is a `string`.  Therefore when the value is set, the validators make sure the value is a string.  If it is not it is converted to a string if possible.  If not a `ValueError` is thrown. 
2. If the `style` is `controlled vocabulary` then the value is checked against `options`.  If `other` is in options that allows other options to be input that are not in the list, kind of a accept anything key.  
3. If a value of None is given the proper None type is set.  If the `style` is a date then the None value for is set to 1980-01-01T00:00:00, or if `list` in `style` the value is set to [].  

When the standards are first read in if `required` is True the value is set to the given default value.  If `required` is False the value is set to the appropriate None value.

In [11]:
extra = {
    'type': float,
    'style': 'number',
    'required': True,
    'units': None,
    'description': 'height',
    'alias': [],
    'options': [],
    'example': 10.0,
    'default': 0.0
}
b.add_base_attribute("height", 0, extra)

In [12]:
b.height = "11.7"
print(b)

base:
	height = 11.7
	temperature = ambient


In [13]:
b.temperature = "fail"

2022-01-10 12:42:49,925 [line 341] mt_metadata.base.metadata.base.__setattr__ - ERROR: fail not found in options list ['ambient', 'air']


MTSchemaError: fail not found in options list ['ambient', 'air']

## A more complicated example

We will look at a more complicated metadata object `mt_metadata.timeseries.Location`

In [14]:
from mt_metadata.timeseries import Location

In [15]:
here = Location()
here.get_attribute_list()

['declination.comments',
 'declination.model',
 'declination.value',
 'elevation',
 'latitude',
 'longitude']

In [16]:
here.attribute_information()

latitude:
	alias: ['lat']
	default: 0.0
	description: Latitude of location in datum specified at survey level.
	example: 23.134
	options: []
	required: True
	style: number
	type: float
	units: degrees
longitude:
	alias: ['lon', 'long']
	default: 0.0
	description: Longitude of location in datum specified at survey level.
	example: 14.23
	options: []
	required: True
	style: number
	type: float
	units: degrees
elevation:
	alias: ['elev']
	default: 0.0
	description: Elevation of location in datum specified at survey level.
	example: 123.4
	options: []
	required: True
	style: number
	type: float
	units: meters
declination.comments:
	alias: []
	default: None
	description: Any comments on declination
	example: estimated from WMM 2016
	options: []
	required: False
	style: free form
	type: string
	units: None
declination.model:
	alias: []
	default: WMM
	description: Geomagnetic reference model used to calculate declination plus the year estimated.
	example: WMM-16
	options: ['WMM', 'EMAG2', 'EM

#### Getting/Setting an attribute

These methods are convenience methods for getting/setting complicated attributes.  For instance getting/setting the declination value from a single call.  This is helpful when filling metadata from a file.  

In [17]:
here.set_attr_from_name("declination.value", 10)
print(here)

location:
	declination.model = WMM
	declination.value = 10.0
	elevation = 0.0
	latitude = 0.0
	longitude = 0.0


In [18]:
here.get_attr_from_name("declination.value")

10.0

In [19]:
# This is the same as
here.declination.value

10.0

## Dictionary

The basic element that the metadata can be in is a Python dictionary with key, value pairs. 

In [20]:
here.to_dict()

{'location': OrderedDict([('declination.model', 'WMM'),
              ('declination.value', 10.0),
              ('elevation', 0.0),
              ('latitude', 0.0),
              ('longitude', 0.0)])}

In [21]:
here.from_dict(
    {
        "location": {
            "declination.value": -11.0,
            "elevation": 759.0,
            "latitude": -34.0,
            "longitude": -104.0
        }
    }
)
print(here)

location:
	declination.model = WMM
	declination.value = -11.0
	elevation = 759.0
	latitude = -34.0
	longitude = -104.0


## JSON

JSON is a standard format human/machine readable and well supported in Python.  There are methods to to read/write JSON files.    

In [22]:
# Compact form
print(here.to_json())

{
    "location": {
        "declination.model": "WMM",
        "declination.value": -11.0,
        "elevation": 759.0,
        "latitude": -34.0,
        "longitude": -104.0
    }
}


In [23]:
here.from_json('{"location": {"declination.model": "WMM", "declination.value": 10.0, "elevation": 99.0, "latitude": 40.0, "longitude": -120.0}}')
print(here)

location:
	declination.model = WMM
	declination.value = 10.0
	elevation = 99.0
	latitude = 40.0
	longitude = -120.0


In [24]:
# Nested form
print(here.to_json(nested=True))

{
    "location": {
        "declination": {
            "model": "WMM",
            "value": 10.0
        },
        "elevation": 99.0,
        "latitude": 40.0,
        "longitude": -120.0
    }
}


In [25]:
here.from_json('{"location": {"declination": {"model": "WMM", "value": -12.0}, "elevation": 199.0, "latitude": 20.0, "longitude": -110.0}}')
print(here)

location:
	declination.model = WMM
	declination.value = -12.0
	elevation = 199.0
	latitude = 20.0
	longitude = -110.0


## XML

XML is also a common format for metadata, though not as human readable.  

In [26]:
print(here.to_xml(string=True))

<?xml version="1.0" ?>
<location>
    <declination>
        <model>WMM</model>
        <value units="degrees">-12.0</value>
    </declination>
    <elevation units="meters">199.0</elevation>
    <latitude units="degrees">20.0</latitude>
    <longitude units="degrees">-110.0</longitude>
</location>



In [27]:
from xml.etree import cElementTree as et
location = et.Element('location')
lat = et.SubElement(location, 'latitude')
lat.text = "-10"
here.from_xml(location)
print(here)

location:
	declination.model = WMM
	declination.value = -12.0
	elevation = 199.0
	latitude = -10.0
	longitude = -110.0


## Pandas Series

Pandas is a common data base object that is commonly used for columnar data.  A series is basically like a single row in a data base. 

In [28]:
pd_series = here.to_series()
print(pd_series)

declination.model      WMM
declination.value    -12.0
elevation            199.0
latitude             -10.0
longitude           -110.0
dtype: object


In [29]:
from pandas import Series

location_series = Series(
    {
        'declination.model': 'WMM',
         'declination.value': -14.0,
         'elevation': 399.0,
         'latitude': -14.0,
         'longitude': -112.0
    }
)

here.from_series(location_series)
print(here)

location:
	declination.model = WMM
	declination.value = -14.0
	elevation = 399.0
	latitude = -14.0
	longitude = -112.0
